# Demonstration Reading DB

In [1]:
import pandas as pd

from db_access.asset_query import AssetQuery
from db_access.db_query import ReadDB
from api.mongo_connection import MongoAPI

In [2]:
# making our api connection
api = MongoAPI('DS4300')

# making our class to construct our queries
reader = ReadDB(api)

The retrieval speeds are very fast to scan over what was 50 mil + rows of data.
However, inorder to best optimize the performance of our api we created a AssetQuery class which saves the results
of the query in memory if the set_save() method is called.
This means that mongo won't have to rerun queries that have been executed before. This is extremely useful when
having to rerun queries in jupyter notebooks

In [3]:
start = pd.Timestamp(year=2015, month=1, day=1)
end = pd.Timestamp(year=2018, month=1, day=1)

# lets get the high, low, open, close, ticker company name and website of a company
# search the database by ticker
# we get back an AssetQuery object which can easity be turned into a pandas DataFrame by using AssetQuery.df
combo_data: AssetQuery = reader.get_asset_data(assets=['AAPL', 'JNJ', 'MMM', 'MSFT', 'AMD', 'NVDA', 'KL', 'L' ,'HD'],
                                         ts_fields=["prccd", "prchd", "prcld", "prcod"], # timeseries fields we want
                                         static_fields=['tic', 'conm', 'weburl'], # static fields
                                         start = start, # start date
                                         end = end, # end date
                                         search_by='tic' # in this query we are using ticker as our asset id
                                         )

In [4]:
# now lets turn the searched data into a DataFrame
# we save a deep copy of the dataframe in the AssetQuery object so we dont have to rerun the query
df_of_combo_data = combo_data.set_save().df
df_of_combo_data.sort_index()


conm             weburl     prccd  \
date       tic                                                         
2015-01-02 AAPL               APPLE INC      www.apple.com  109.3300   
           AMD   ADVANCED MICRO DEVICES        www.amd.com    2.6900   
           HD            HOME DEPOT INC  www.homedepot.com  103.4300   
           JNJ        JOHNSON & JOHNSON        www.jnj.com  104.5200   
           KL    KIRKLAND LAKE GOLD LTD     www.klgold.com    3.0004   
...                                 ...                ...       ...   
2017-12-29 KL    KIRKLAND LAKE GOLD LTD     www.klgold.com   19.2700   
           L                 LOEWS CORP      www.loews.com   50.0300   
           MMM                    3M CO         www.3m.com  235.3700   
           MSFT          MICROSOFT CORP  www.microsoft.com   85.5400   
           NVDA             NVIDIA CORP     www.nvidia.com  193.5000   

                    prchd     prcld     prcod  
date       tic                                 
2015-01-02 AAPL  111.4400  107.3500  111.3900  
           AMD     2.7300    2.6400    2.7000  
           HD    105.6548  102.6700  105.1600  
           JNJ   105.5500  104.1300  105.0500  
           KL      3.0740    2.9902    3.0475  
...                   ...       ...       ...  
2017-12-29 KL     19.3500   18.8700   19.1800  
           L      50.3000   50.0000   50.2000  
           MMM   237.3100  235.3700  236.4800  
           MSFT   86.0500   85.5000   85.6300  
           NVDA  198.4600  193.5000  198.4600  

[7547 rows x 6 columns]

### Can also pull data for static data only query's

In [5]:
static_data: AssetQuery = reader.get_asset_data(assets=['AAPL', 'JNJ', 'MMM', 'MSFT', 'AMD', 'NVDA', 'KL', 'L' ,'HD'],
                                         static_fields=['tic', 'conm', 'weburl', 'cusip'], # static fields
                                         search_by='tic' # in this query we are using ticker as our asset id
                                         )

In [6]:
# now lets turn the searched data into a DataFrame
df_of_static_data = static_data.set_save().df
df_of_static_data.sort_index()

,cusip,conm,weburl
tic,,,
AAPL,037833100,APPLE INC,www.apple.com
AMD,007903107,ADVANCED MICRO DEVICES,www.amd.com
HD,437076102,HOME DEPOT INC,www.homedepot.com
JNJ,478160104,JOHNSON & JOHNSON,www.jnj.com
KL,49741E100,KIRKLAND LAKE GOLD LTD,www.klgold.com
L,540424108,LOEWS CORP,www.loews.com
MMM,88579Y101,3M CO,www.3m.com
MSFT,594918104,MICROSOFT CORP,www.microsoft.com
NVDA,67066G104,NVIDIA CORP,www.nvidia.com


In [7]:
static_data_cuisp: AssetQuery = reader.get_asset_data(
     assets=['037833100', '007903107', '437076102', '478160104', '49741E100', '540424108', '88579Y101',
             '594918104', '67066G104'],
     static_fields=['tic', 'conm', 'weburl'], # static fields
     search_by='cusip' # in this query we are using ticker as our asset id
     )

In [8]:
# now lets turn the searched data into a DataFrame
df_of_static_data_cusip = static_data_cuisp.set_save().df
df_of_static_data_cusip.sort_index()

,tic,conm,weburl
cusip,,,
007903107,AMD,ADVANCED MICRO DEVICES,www.amd.com
037833100,AAPL,APPLE INC,www.apple.com
437076102,HD,HOME DEPOT INC,www.homedepot.com
478160104,JNJ,JOHNSON & JOHNSON,www.jnj.com
49741E100,KL,KIRKLAND LAKE GOLD LTD,www.klgold.com
540424108,L,LOEWS CORP,www.loews.com
594918104,MSFT,MICROSOFT CORP,www.microsoft.com
67066G104,NVDA,NVIDIA CORP,www.nvidia.com
88579Y101,MMM,3M CO,www.3m.com


In [9]:
# brew services restart mongodb-community
